In [ ]:
!pip install -q datasets
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `QA-dataset` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `QA-dataset`


In [ ]:
import pandas as pd
from datasets import Dataset

# Step 1: Download the file manually using pandas
url = "https://huggingface.co/datasets/Prathamesh25/QA-textbook-dataset/resolve/main/qa_dataset.jsonl"
df = pd.read_json(url, lines=True)

# Step 2: Convert to Hugging Face dataset
dataset = Dataset.from_pandas(df)

# Step 3: Shuffle and limit
dataset = dataset.shuffle(seed=42).select(range(409))

# Step 4: Apply your transformation
def transform_conversation(example):
    segments = example['text'].split('###')
    reformatted_segments = []
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')
    return {'text': ''.join(reformatted_segments)}

# Step 5: Map the transformation
transformed_dataset = dataset.map(transform_conversation)


Map:   0%|          | 0/409 [00:00<?, ? examples/s]

In [ ]:
transformed_dataset.push_to_hub("qa-textbook-transformed-dataset")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]